# Notebook 3 : SQL

In [53]:
# Décommenter la ligne suivante pour installer ibis
# %pip install 'ibis-framework[sqlite]'

In [54]:
import sqlite3

import pandas as pd
import ibis

from ibis import _

ibis.options.interactive = True

query_tables = "SELECT name FROM sqlite_master WHERE type='table'"

## STAR

Nous considérons les données des stations de vélos en libre service [STAR](https://www.star.fr/) de Rennes Métropole. Une copie de la base SQLite est disponible dans le fichier `star.db`. Nous utilisons d'abord Pandas pour répondre aux questions, puis Ibis.

1. Se connecter à la base de données et afficher la liste des tables à l'aide de la fonction `read_sql` de Pandas et de la requête `query_tables`.

In [55]:
con = sqlite3.connect("data/star.db")
tables = pd.read_sql(query_tables, con)
print(tables)

        name
0  Topologie
1       Etat


2. Récupérer le contenu de la table `Etat` dans un dataframe et afficher la liste des variables disponibles. Même question pour la table `Topologie`.

In [56]:
etat_df = pd.read_sql("SELECT * FROM Etat", con)
print(etat_df.dtypes)
topologie_df = pd.read_sql("SELECT * FROM Topologie", con)
print(topologie_df.dtypes)

id                            int64
nom                          object
latitude                    float64
longitude                   float64
etat                         object
nb_emplacements               int64
emplacements_disponibles      int64
velos_disponibles             int64
date                        float64
data                         object
dtype: object
id                     int64
nom                   object
adresse_numero        object
adresse_voie          object
commune               object
latitude             float64
longitude            float64
id_correspondance    float64
mise_en_service      float64
nb_emplacements        int64
id_proche_1            int64
id_proche_2            int64
id_proche_3            int64
terminal_cb           object
dtype: object


3. Sélectionner l'identifiant `id`, le nom `nom` et l'identifiant de la station la plus proche `id_proche_1` depuis la table `Topologie`.

In [57]:
print(topologie_df[["id", "nom", "id_proche_1"]])

    id                nom  id_proche_1
0    1         République            2
1    2             Mairie            1
2    3      Champ Jacquet            2
3   10   Musée Beaux-Arts           12
4   12                TNB           10
..  ..                ...          ...
78  62         Clemenceau           63
79  66  Bréquigny Piscine           65
80  69    Champs Manceaux           66
81  85       La Courrouze           20
82  86          Armorique           78

[83 rows x 3 columns]


4. Faire une jointure sur la table précédente pour créer une table qui contient la liste des stations avec l'identifiant, le nom et le nom de la station la plus proche associée à l'identifiant `id_proche_1`. Les variables utilisées comme clés sont différents, penser à utiliser les arguments `left_on` et `right_on` de la méthode `merge`.

In [58]:
print(
    topologie_df[["id", "nom", "id_proche_1"]]
    .merge(topologie_df, left_on="id_proche_1", right_on="id", how="left")
    .filter(items=["id_x", "nom_x", "nom_y"])
    .rename(columns={"id_x": "id", "nom_x": "nom", "nom_y": "nom_proche_1"})
)

    id                nom       nom_proche_1
0    1         République             Mairie
1    2             Mairie         République
2    3      Champ Jacquet             Mairie
3   10   Musée Beaux-Arts                TNB
4   12                TNB   Musée Beaux-Arts
..  ..                ...                ...
78  62         Clemenceau     Henri Fréville
79  66  Bréquigny Piscine                NaN
80  69    Champs Manceaux  Bréquigny Piscine
81  85       La Courrouze     Pont de Nantes
82  86          Armorique         Gros-Chêne

[83 rows x 3 columns]


5. Ajouter à la table précédente la distance entre la station et la station la plus proche.

In [59]:
print(
    topologie_df
    .merge(topologie_df, left_on="id_proche_1", right_on="id", suffixes=("_x", "_y"))
    .assign(distance=lambda row:
        (
            (row.latitude_x - row.latitude_y)**2
            + (row.longitude_x - row.longitude_y)**2
        )**0.5
    )
    .filter(items=["id_x", "nom_x", "nom_y", "distance"])
    .rename(columns={"id_x": "id", "nom_x": "nom", "nom_y": "nom_proche_1"})
)

    id                nom        nom_proche_1  distance
0    1         République              Mairie  0.001753
1    3      Champ Jacquet              Mairie  0.001733
2    9      Saint-Georges              Mairie  0.004406
3    2             Mairie          République  0.001753
4   10   Musée Beaux-Arts                 TNB  0.001890
..  ..                ...                 ...       ...
76  11            Liberté          Les Halles  0.001925
77  19    Plélo Colombier          Les Halles  0.002156
78  24  Place de Bretagne  Office de Tourisme  0.001135
79  55         Préfecture             Cucillé  0.004111
80  69    Champs Manceaux   Bréquigny Piscine  0.008096

[81 rows x 4 columns]


6. Créer une table avec le nom des trois stations les plus proches du point GPS *(48.1179151,-1.7028661)* classées par ordre de distance et le nombre de vélos disponibles dans ces stations.

In [60]:
print(
    etat_df
    .assign(distance=
        (
            (48.1179151 - etat_df.latitude)**2
            + (-1.7028661 - etat_df.longitude)**2
        )**0.5
    )
    .filter(items=["id", "nom", "distance", "velos_disponibles" ])
    .sort_values(by="distance", ascending=True)
    .head(3)
)

    id                   nom  distance  velos_disponibles
50  56                Berger  0.002746                 10
17  52  Villejean-Université  0.003401                 11
74  38               Marbeuf  0.006216                  9


7. Reprendre les questions précédentes en utilisant le module `ibis`. Pour les jointures, utiliser la méthode `left_join`.

In [61]:
con_ibis = ibis.sqlite.connect("data/star.db")
con_ibis.tables

Tables
------
- Etat
- Topologie

In [62]:
etat_table = con_ibis.table("Etat")
print(etat_table.columns)

('id', 'nom', 'latitude', 'longitude', 'etat', 'nb_emplacements', 'emplacements_disponibles', 'velos_disponibles', 'date', 'data')


In [64]:
topologie_table = con_ibis.table("Topologie")
print(topologie_table.columns)

('id', 'nom', 'adresse_numero', 'adresse_voie', 'commune', 'latitude', 'longitude', 'id_correspondance', 'mise_en_service', 'nb_emplacements', 'id_proche_1', 'id_proche_2', 'id_proche_3', 'terminal_cb')


In [65]:
print(
    topologie_table.select("id", "nom", "id_proche_1")
)

┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ id    ┃ nom                ┃ id_proche_1 ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ int64 │ string             │ int64       │
├───────┼────────────────────┼─────────────┤
│     1 │ République         │           2 │
│     2 │ Mairie             │           1 │
│     3 │ Champ Jacquet      │           2 │
│    10 │ Musée Beaux-Arts   │          12 │
│    12 │ TNB                │          10 │
│    14 │ Laënnec            │          35 │
│    17 │ Charles de Gaulle  │          16 │
│    20 │ Pont de Nantes     │          43 │
│    22 │ Oberthur           │          44 │
│    25 │ Office de Tourisme │          24 │
│     … │ …                  │           … │
└───────┴────────────────────┴─────────────┘


In [68]:
print(
topologie_table
    .left_join(
        etat_table,
        topologie_table.id_proche_1 == etat_table.id)
    .select("id", "nom", "nom_right")
    .rename(nom_proche_1="nom_right")
)

┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ id    ┃ nom                ┃ nom_proche_1       ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ int64 │ string             │ string             │
├───────┼────────────────────┼────────────────────┤
│     1 │ République         │ Mairie             │
│     2 │ Mairie             │ République         │
│     3 │ Champ Jacquet      │ Mairie             │
│    10 │ Musée Beaux-Arts   │ TNB                │
│    12 │ TNB                │ Musée Beaux-Arts   │
│    14 │ Laënnec            │ Pont de Châteaudun │
│    17 │ Charles de Gaulle  │ Champs Libres      │
│    20 │ Pont de Nantes     │ Cité Judiciaire    │
│    22 │ Oberthur           │ Metz - Sévigné     │
│    25 │ Office de Tourisme │ Place de Bretagne  │
│     … │ …                  │ …                  │
└───────┴────────────────────┴────────────────────┘


In [71]:
print(
topologie_table
    .left_join(
        etat_table,
        topologie_table.id_proche_1 == etat_table.id)
    .mutate(
        distance = ((_.latitude - _.latitude_right) * (_.latitude - _.latitude_right) + (_.longitude - _.longitude_right) * (_.longitude - _.longitude_right))**0.5
    )
    .select("id", "nom", "nom_right","distance")
    .rename(nom_proche_1="nom_right")
)

┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ id    ┃ nom                ┃ nom_proche_1       ┃ distance ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ int64 │ string             │ string             │ float64  │
├───────┼────────────────────┼────────────────────┼──────────┤
│     1 │ République         │ Mairie             │ 0.001753 │
│     2 │ Mairie             │ République         │ 0.001753 │
│     3 │ Champ Jacquet      │ Mairie             │ 0.001733 │
│    10 │ Musée Beaux-Arts   │ TNB                │ 0.001890 │
│    12 │ TNB                │ Musée Beaux-Arts   │ 0.001890 │
│    14 │ Laënnec            │ Pont de Châteaudun │ 0.003682 │
│    17 │ Charles de Gaulle  │ Champs Libres      │ 0.002824 │
│    20 │ Pont de Nantes     │ Cité Judiciaire    │ 0.002758 │
│    22 │ Oberthur           │ Metz - Sévigné     │ 0.004351 │
│    25 │ Office de Tourisme │ Place de Bretagne  │ 0.001135 │
│     … │ …                  │ …                  │    

In [110]:
print(
    etat_table
    .mutate(
        distance = (
            (48.1179151 - etat_table.latitude)*(48.1179151 - etat_table.latitude) 
            + (-1.7028661 - etat_table.longitude)*(-1.7028661 - etat_table.longitude)
            )**0.5
    )
    .order_by("distance")
    .limit(3)
    .select("nom", "distance", "velos_disponibles")
)

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ nom                  ┃ distance ┃ velos_disponibles ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ string               │ float64  │ int64             │
├──────────────────────┼──────────┼───────────────────┤
│ Berger               │ 0.002746 │                10 │
│ Villejean-Université │ 0.003401 │                11 │
│ Marbeuf              │ 0.006216 │                 9 │
└──────────────────────┴──────────┴───────────────────┘


8. (*Bonus*) Écrire des requêtes SQL pour obtenir les résultats demandés dans les questions 3 à 6. La fonction `to_sql` pourra être utilisée pour de l'aide.

## Musique

Le dépôt GitHub [lerocha/chinook-database](https://github.com/lerocha/chinook-database) met à disposition des bases de données de bibliothèques musicales. Une copie de la base SQLite est disponible dans le fichier `chinook.db`.

1. Utiliser le module `ibis` pour vous connecter à la base de données et explorer les tables formant le jeu de données pour le découvrir. En particulier, remarquer comment les tables `Playlist`, `PlaylistTrack` et `Track` sont liées entre elles.

In [111]:
con_chinook = ibis.sqlite.connect("data/chinook.db")
con_chinook.tables

Tables
------
- Album
- Artist
- Customer
- Employee
- Genre
- Invoice
- InvoiceLine
- MediaType
- Playlist
- PlaylistTrack
- Track

In [114]:
playlist = con_chinook.table("Playlist")
playlist

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ PlaylistId ┃ Name         ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ !int64     │ string(120)  │
├────────────┼──────────────┤
│          1 │ Music        │
│          2 │ Movies       │
│          3 │ TV Shows     │
│          4 │ Audiobooks   │
│          5 │ 90’s Music   │
│          6 │ Audiobooks   │
│          7 │ Movies       │
│          8 │ Music        │
│          9 │ Music Videos │
│         10 │ TV Shows     │
│          … │ …            │
└────────────┴──────────────┘

In [115]:
track = con_chinook.table("Track")
track

┏━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ TrackId ┃ Name                                    ┃ AlbumId ┃ MediaTypeId ┃ GenreId ┃ Composer                                                               ┃ Milliseconds ┃ Bytes    ┃ UnitPrice       ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ !int64  │ !string(200)                            │ int64   │ !int64      │ int64   │ string(220)                                                            │ !int64       │ int64    │ !decimal(10, 2) │
├─────────┼─────────────────────────────────────────┼─────────┼─────────────┼─────────┼────────────────────────────────────────────────────────────────────────┼──────────────┼──────────┼─────────────────┤
│       1 │ For Those About To Rock (We Salute You) │       1 │           1 │       1 │ Angus Young, Malcolm Young, Brian Johnson                              │       343719 │ 11170334 │            0.99 │
│       2 │ Balls to the Wall                       │       2 │           2 │       1 │ NULL                                                                   │       342562 │  5510424 │            0.99 │
│       3 │ Fast As a Shark                         │       3 │           2 │       1 │ F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman                    │       230619 │  3990994 │            0.99 │
│       4 │ Restless and Wild                       │       3 │           2 │       1 │ F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman │       252051 │  4331779 │            0.99 │
│       5 │ Princess of the Dawn                    │       3 │           2 │       1 │ Deaffy & R.A. Smith-Diesel                                             │       375418 │  6290521 │            0.99 │
│       6 │ Put The Finger On You                   │       1 │           1 │       1 │ Angus Young, Malcolm Young, Brian Johnson                              │       205662 │  6713451 │            0.99 │
│       7 │ Let's Get It Up                         │       1 │           1 │       1 │ Angus Young, Malcolm Young, Brian Johnson                              │       233926 │  7636561 │            0.99 │
│       8 │ Inject The Venom                        │       1 │           1 │       1 │ Angus Young, Malcolm Young, Brian Johnson                              │       210834 │  6852860 │            0.99 │
│       9 │ Snowballed                              │       1 │           1 │       1 │ Angus Young, Malcolm Young, Brian Johnson                              │       203102 │  6599424 │            0.99 │
│      10 │ Evil Walks                              │       1 │           1 │       1 │ Angus Young, Malcolm Young, Brian Johnson                              │       263497 │  8611245 │            0.99 │
│       … │ …                                       │       … │           … │       … │ …                                                                      │            … │        … │               … │
└─────────┴─────────────────────────────────────────┴─────────┴─────────────┴─────────┴────────────────────────────────────────────────────────────────────────┴──────────────┴──────────┴─────────────────┘

In [116]:
playlist_track = con_chinook.table("PlaylistTrack")
playlist_track

┏━━━━━━━━━━━━┳━━━━━━━━━┓
┃ PlaylistId ┃ TrackId ┃
┡━━━━━━━━━━━━╇━━━━━━━━━┩
│ !int64     │ !int64  │
├────────────┼─────────┤
│          1 │    3402 │
│          1 │    3389 │
│          1 │    3390 │
│          1 │    3391 │
│          1 │    3392 │
│          1 │    3393 │
│          1 │    3394 │
│          1 │    3395 │
│          1 │    3396 │
│          1 │    3397 │
│          … │       … │
└────────────┴─────────┘

2. Quelles sont les playlists qui contiennent le plus de pistes ?

In [120]:
(
    playlist_track
    .group_by("PlaylistId")
    .aggregate(n_track=playlist_track.count())
    .left_join(
        playlist, "PlaylistId"
    )
    .select("PlaylistId", "Name", "n_track")
    .order_by(
        ibis.desc("n_track")
    )
)

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ PlaylistId ┃ Name                       ┃ n_track ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ int64      │ string(120)                │ int64   │
├────────────┼────────────────────────────┼─────────┤
│          1 │ Music                      │    3290 │
│          8 │ Music                      │    3290 │
│          5 │ 90’s Music                 │    1477 │
│          3 │ TV Shows                   │     213 │
│         10 │ TV Shows                   │     213 │
│         12 │ Classical                  │      75 │
│         11 │ Brazilian Music            │      39 │
│         17 │ Heavy Metal Classic        │      26 │
│         13 │ Classical 101 - Deep Cuts  │      25 │
│         14 │ Classical 101 - Next Steps │      25 │
│          … │ …                          │       … │
└────────────┴────────────────────────────┴─────────┘

3. Construire une table contenant les informations suivantes sur la playlist `Classical` : le titre de chaque piste ainsi que le titre de l'album dont cette piste est tirée.

In [121]:
album = con_chinook.table("Album")

print(
    playlist_track
    .left_join(playlist, playlist_track.PlaylistId == playlist.PlaylistId)
    .rename(PlaylistName="Name")
    .filter(_.PlaylistName == "Classical")
    .left_join(track, _.TrackId == track.TrackId)
    .left_join(album, _.AlbumId == album.AlbumId)
    .select("Name", "Title")
)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ Name                                                                 ┃ Title                                     
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ string(200)                                                          │ string(160)                               
├──────────────────────────────────────────────────────────────────────┼───────────────────────────────────────────
│ Intoitus: Adorate Deum                                               │ Adorate Deum: Gregorian Chant from the Pro
│ Miserere mei, Deus                                                   │ Allegri: Miserere                         
│ Canon and Gigue in D Major: I. Canon                                 │ Pachelbel: Canon & Gigue                  
│ Concerto No. 1 in E Major, RV 269 "Spring": I. Allegro               │

4. (*Bonus*) Écrire une requête SQL donnant le résultat de la question précédente. La fonction `to_sql` pourra être utilisée pour de l'aide.